# Birefringence calibration test

This script implements the method of [\[Belendez et al. 2009\]](https://physlab.lums.edu.pk/images/3/30/Cellophane2.pdf) to acquire the birefringence value of a birefringent material (tape) based on a series of measurements under different single-wavelength illuminants. This is a test verifying that our method can acquire a correct birefringence value under "ideal" conditions and assumptions.

Units:  
angles: degrees
wavelengths: nanometers  
thickness: nanometers  

Types of tape:  
Sure Start: 2.6 mil / 66040 nm thick  
Heavy Duty: 3.1 mil / 78740 nm thick  

Our laser wavelengths:  
650 nm (red)  
532 nm (green)  
405 nm (purple)

In [1]:
%cd ".."

import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import sensor_data
import color_utils
import tape_data
from experiment import *

/Users/kate/diy-hyperspectral


In [2]:
# Laser params
laser_wavelengths = np.array([405.0, 532.0, 650.0])

# Tape params
birefringence = 0.0077
thickness = tape_data.THICKNESS_SS # Sure Start
# thickness = tape_data.THICKNESS_HD # Heavy Duty

In [3]:
configs = []

# Create filters from waveplates and analyzer angles
theta = np.linspace(0.0, 180.0, 37)
alpha = np.repeat(45.0, theta.shape[0])
wp = Waveplate(thickness, birefringence)

for a in range(theta.shape[0]):
    configs.append(Config(alpha[a], theta[a], [wp]))

configs = np.array(configs)

In [4]:
# Run experiments - measure response for every wavelength and filter configuration
sensor_wavelengths = sensor_data.NIKON_WAVELENGTHS
sensor_response = sensor_data.TEST_RESPONSE

measurements = np.zeros((len(laser_wavelengths),len(configs),3))

for w, wavelength in enumerate(laser_wavelengths):
    filters = np.array([config.transmission(wavelength) for config in configs])
    filters = np.expand_dims(filters, axis = 1)
    measurements[w,:,:] = measure_single(wavelength, 1.0, filters, sensor_response, sensor_wavelengths)

/Users/kate/diy-hyperspectral/polarization.py:75: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [5]:
# Solve for gamma (retardance angle) for each wavelength

x = np.cos(np.deg2rad(2 * theta))
A = np.vstack([x, np.ones(x.shape[0])]).T

cos_gamma = np.zeros((len(laser_wavelengths),3))

for w in range(len(laser_wavelengths)):
    for c in range(3):
        # Solve Eq. 16 by least squares
        m, n = np.linalg.lstsq(A, measurements[w,:,c], rcond=None)[0]
        
        # Solve Eq. 20
        cos_gamma[w,c] = m / n

# m / n should be the same for all color channels, take the average
cos_gamma = np.average(cos_gamma, axis = 1)

# Clamp to [-1, 1]
cos_gamma = np.clip(cos_gamma, -1.0, 1.0)

# Find smallest possible gamma
gamma_min = np.arccos(cos_gamma)

# Resolve ambiguity in gamma (see Eqs. 21-24)
wav_blue_over_wav_j = laser_wavelengths[0] / laser_wavelengths
print("wav_blue_over_wav_j: ", wav_blue_over_wav_j)
print()

# Solve for N and +/- per wavelength
# Choose option that gives wav_blue_over_wav_j = gamma_j_over_gamma_blue per wavelength
options_str = ['gamma_min', 
               '2 pi - gamma_min',
               '2 pi + gamma_min',
               '4 pi - gamma_min',
               '4 pi + gamma_min'] # etc.

options = [gamma_min, 
           2. * np.pi - gamma_min,
           2. * np.pi + gamma_min,
           4. * np.pi - gamma_min,
           4. * np.pi + gamma_min] # etc.

for i, gamma_blue in enumerate(options):
    print()
    print('Possible solutions if gamma_blue = ', options_str[i])
    
    for j, gamma_j in enumerate(options):
        # Skip all options where gamma_j > gamma_blue (not a possible solution)
        if j > i:
            continue
            
        gj_gb = gamma_j / gamma_blue[0]
        
        # Replace all options where gamma_j_over_gamma_blue > 1 with 0. (not a possible solution)
        # Because we can assume gamma is inversely proportional to wavelength (by Eq. 1)
        gj_gb = np.where(gj_gb > 1.0, 0., gj_gb)
        
        # gamma_blue must equal gamma_j if j = blue
        if i != j:
            gj_gb[0] = 0.
        
        print('gamma_j = ', options_str[j])
        print(gj_gb)
        
# Choose best options, verify that gamma_blue > gamma_green > gamma_red
blue_opt  = 2
green_opt = 1
red_opt   = 1

gamma = np.zeros_like(gamma_min)
gamma[0] = options[blue_opt][0] # blue
gamma[1] = options[green_opt][1] # green
gamma[2] = options[red_opt][2] # red

print()
print("Final gamma: ", gamma)

wav_blue_over_wav_j:  [1.         0.7612782  0.62307692]


Possible solutions if gamma_blue =  gamma_min
gamma_j =  gamma_min
[1.         0.17277842 0.85172547]

Possible solutions if gamma_blue =  2 pi - gamma_min
gamma_j =  gamma_min
[0.         0.05931815 0.29241373]
gamma_j =  2 pi - gamma_min
[1. 0. 0.]

Possible solutions if gamma_blue =  2 pi + gamma_min
gamma_j =  gamma_min
[0.         0.03516945 0.17337072]
gamma_j =  2 pi - gamma_min
[0.         0.76127819 0.62307692]
gamma_j =  2 pi + gamma_min
[1.         0.8316171  0.96981837]

Possible solutions if gamma_blue =  4 pi - gamma_min
gamma_j =  gamma_min
[0.         0.02531373 0.12478613]
gamma_j =  2 pi - gamma_min
[0.         0.54794117 0.44846877]
gamma_j =  2 pi + gamma_min
[0.         0.59856863 0.69804103]
gamma_j =  4 pi - gamma_min
[1. 0. 0.]

Possible solutions if gamma_blue =  4 pi + gamma_min
gamma_j =  gamma_min
[0.         0.01957722 0.09650753]
gamma_j =  2 pi - gamma_min
[0.         0.42376865 0.34683834]
gamma_

In [6]:
# Solve for birefringence

inv_wav = 1.0 / laser_wavelengths
A = np.vstack((inv_wav, np.zeros(inv_wav.shape))).T

# Solve Eq. 1 by least squares
bir_thk_2pi, _ = np.linalg.lstsq(A, gamma, rcond=None)[0]

bir_thk = bir_thk_2pi / (2. * np.pi)
birefringence_meas = bir_thk / thickness

print(birefringence_meas)

# Calculate birefringence separately per wavelength instead
birefringence_meas2 = gamma * laser_wavelengths / (2. * np.pi * thickness)
print(birefringence_meas2)

0.007699999992650467
[0.0077 0.0077 0.0077]


In [7]:
print(np.rad2deg(gamma))

# Double check gammas with measured birefringence
gamma_exp = (2. * np.pi) * birefringence_meas * thickness / laser_wavelengths
print(np.rad2deg(gamma_exp))

[452.0071111  344.10315684 281.63519994]
[452.00711068 344.10315757 281.63519973]
